<a href="https://colab.research.google.com/github/2019mohamed/ArabDialectClassification/blob/master/MLP_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/two_clo_dialect_dataset.csv')
df

,new_text,label
0,بالنهايه ينتفض يغير,0
1,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...,0
2,مبين كلامه خليجي,0
3,يسلملي مرورك وروحك الحلوه,0
4,وين الغيبه اخ محمد,0
...,...,...
458192,مبسوطين منك اللي باسطانا,17
458193,والله ماينده ابش يختي,17
458194,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...,17
458195,الله يبارك وبالعافيه,17


In [ ]:
corpus = df['new_text'].astype('str').tolist()
labels = df['label'].tolist()

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler , BatchSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import pickle
from scipy.sparse import coo_matrix

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('GPU')
else:
  device = torch.device('cpu')
  print('CPU')

GPU


In [ ]:
vec = pickle.load(open('/content/drive/MyDrive/vectorizer.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
def get_index ():
  freqs = df['label'].value_counts()

  start = [0 for _ in range(18)]
  start[0] = 0
  for c in range(1,18):
      start[c] += freqs[c-1] + start[c-1]  
  indexes = []# (mini , maxi+1) for all classes
  for c in range(18):
      indexes.append(  (start[c] , start[c] + freqs[c]) )
    
  return indexes

In [ ]:
def sample_data (data , label ,intervals, lim = 1000):
  data = np.array(data)
  label = np.array(label)
  ret = np.array([])
  retl = np.array([])
  for i in range(18):
    l = intervals[i][0]
    h = intervals[i][1]
    index = np.random.choice(range(l,h), size = lim , replace = False )
    ret = np.concatenate((ret , data[index]), axis = 0)
    retl = np.concatenate((retl , label[index]), axis = 0)

  return ret,retl

In [ ]:
indexes = get_index()

In [ ]:
import numpy as np

In [ ]:
def get_best_features (x , k = 5000):
  #we can do valence score, but, for simplicty doing that ...
  n = x.shape[0]
  m = x.shape[1]
  sum_of_feat = []
  for i in range(m):
    sum_of_feat.append(np.sum(x[:,i]))

  sum_of_feat = np.array(sum_of_feat)
  indexs = sum_of_feat.argsort()[-k:]
  return indexs

In [ ]:
class SeqData (Dataset):
    def __init__(self):
      self.sample_corpus, self.sample_label = sample_data(corpus , labels, indexes , lim = 1000 )
      self.X = vec.transform(self.sample_corpus)
      self.important_indexs = get_best_features(self.X)
      self.X = self.X[:,self.important_indexs]
      self.cooX = coo_matrix(self.X) 
      self.tensor_X = torch.sparse_coo_tensor([self.cooX.row , self.cooX.col], self.cooX.data , dtype = torch.float) 
      self.dim = self.tensor_X.shape[1]
    def __len__(self):
      return len(self.sample_label)
    def __getitem__(self, index):
      return self.tensor_X[index] , self.sample_label[index]


In [ ]:
import numpy as np

In [ ]:
def split_rand(dataset,batch_size, split_ratio=0.7, seed=42, shuffle=True):
    import math
    num_entries = len(dataset)
    indices = list(range(num_entries))
    np.random.seed(seed)
    np.random.shuffle(indices)
    split = int(math.floor(split_ratio * num_entries))
    train_idx, valid_idx = indices[:split], indices[split:]
    
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(
            dataset, sampler=train_sampler,
            batch_size=batch_size)
    
    valid_loader = DataLoader(
            dataset, sampler=valid_sampler,
            batch_size=batch_size)

    return train_loader, valid_loader

In [ ]:
def train(net, trainloader, optimizer, criterion):
    net.train()

    running_loss = 0
    total_iters = len(trainloader)

    for idx , data  in enumerate(trainloader):

        x, labels= data
        labels = labels.type(torch.LongTensor)
        x = x.to(device)
        labels = labels.to(device)
        #print(x)
        outputs = net(x)

        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss = running_loss / total_iters

    return running_loss

In [ ]:
def eval_net(net, dataloader, criterion):
    net.eval()

    total = 0
    total_loss = 0
    total_correct = 0

    for idx ,  data in enumerate(dataloader):
        x, labels = data
        labels = labels.type(torch.LongTensor)
        
        x = x.to(device)
        labels = labels.to(device)

        total += len(labels)
        outputs = net(x.float())
        _, predicted = torch.max(outputs.data, 1)

        total_correct += (predicted == labels.data).sum().item()
        loss = criterion(outputs, labels)
        total_loss += loss.item() * len(labels)

    loss, acc = 1.0*total_loss / total, 1.0*total_correct / total


    return loss, acc

In [ ]:
class MLP(nn.Module):
    """MLP with linear output"""
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        """MLP layers construction
        Paramters
        ---------
        num_layers: int
            The number of linear layers
        input_dim: int
            The dimensionality of input features
        hidden_dim: int
            The dimensionality of hidden units at ALL layers
        output_dim: int
            The number of classes for prediction
        """
        super(MLP, self).__init__()
        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers
        self.output_dim = output_dim

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            #for layer in range(num_layers - 1):
             #   self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for i in range(self.num_layers - 1):
                h = F.relu(self.linears[i](h))
            return F.relu(self.linears[-1](h))

In [ ]:
dataset = SeqData()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [ ]:
train_loader , test_loader = split_rand(dataset, batch_size = 16)

In [ ]:
dim = dataset.dim
dim

5000

In [ ]:
import_i = dataset.important_indexs
pickle.dump(import_i , open('important_indexs.pkl','wb'))
import_i

array([279614, 300436,  53293, ...,  63214,  50730,  50885])

In [ ]:
ii = pickle.load(open('important_indexs.pkl', 'rb'))
ii

array([279614, 300436,  53293, ...,  63214,  50730,  50885])

In [ ]:
model = MLP(2 ,dim ,1000,18).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss = []
test_loss = []
test_acc = []

In [ ]:
for _ in range(10):
    tloss = train(model, train_loader, optimizer, criterion)
    train_loss.append(tloss)
    print(tloss)
    
    eloss, eacc= eval_net(model, test_loader, criterion)
    test_loss.append(eloss)
    test_acc.append(eacc)
    print(eloss,' ',eacc)

2.7800031591793
2.7475320318010117   0.13037037037037036
2.6380540698312864
2.7626448263945402   0.13148148148148148
2.564096712370209
2.7884154874307137   0.13444444444444445
2.5278112935233237
2.808024442460802   0.13425925925925927
2.4742248115805805
2.8226112831963435   0.13314814814814815
2.437658878601142
2.8652142927381727   0.13666666666666666
2.4257110081953446
2.8723501279619006   0.13074074074074074
2.4195674114723498
2.9144906315980132   0.13203703703703704
2.4163304593054775
2.916302052250615   0.12481481481481481
2.4162758699225897
2.8952039082845054   0.12703703703703703


In [ ]:
torch.save(model, 'MLP3.pth')

In [ ]:
the_model = torch.load('MLP3.pth')

In [ ]:
torch.save(model.state_dict(), 'MLP4.pth')

In [ ]:
the_model1 = MLP(2 ,dim ,1000,18)
the_model1.load_state_dict(torch.load('/content/drive/MyDrive/MLP4.pth'))

<All keys matched successfully>

In [ ]:
the_model1.eval()

MLP(
  (linears): ModuleList(
    (0): Linear(in_features=5000, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=18, bias=True)
  )
  (batch_norms): ModuleList()
)

In [ ]:
input = torch.randint(0,10,(1,5000))
print(input)


tensor([[1, 0, 7,  ..., 0, 3, 5]])


In [ ]:
output = the_model1(input.float())
output

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       grad_fn=<ReluBackward0>)

In [ ]:
_, predicted = torch.max(output.data, 1)
predicted

tensor([0])

Now, we will code the API for the model using flask framework

In [ ]:
!pip install tashaphyne

     |████████████████████████████████| 251 kB 12.2 MB/s 
     |████████████████████████████████| 126 kB 43.6 MB/s 


In [ ]:
!pip install pyarabic

In [ ]:
!pip install emoji 

     |████████████████████████████████| 175 kB 12.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=c8668cd7a88f3a516259c5f6c0a822bfc8101e91d9771c936a3157d630aca64c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Cleaning data

from nltk.corpus import stopwords
from textblob import TextBlob
from tashaphyne.stemming import ArabicLightStemmer
#from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
import re
import emoji


stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}
ArListem = ArabicLightStemmer()

def stem(text):
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)

def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

def extract_hashtag(text):
    
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list :
        word_list.extend(split_hashtag_to_words(word))
    return word_list

def split_hashtag_to_words(tag):
    tag = tag.replace('#','')
    tags = tag.split('_')
    if len(tags) > 1 :
        
        return tags
    pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")
    return pattern.findall(tag)

def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)

def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False
    
def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U00002700-\U000027BF"  # Dingbats
                                   u"\U00002600-\U000026FF"  # Miscellaneous Symbols
                                   u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                   u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = emoji.demojize(text)
    return text
  
def clean_tweet(text):
    text = re.sub('#\d+K\d+', ' ', text)  # years like 2K19
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('@[^\s]+',' ',text)
    text = clean_hashtag(text)
    text = remove_emoji(text)
    return text


    
def clean_text (text):
    ## Clean for tweets
    text = clean_tweet(text)
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    ## Remove stop words
    text = remove_stop_words(text)
    ## Remove numbers
    text = re.sub("\d+", " ", text)
    ## Remove Tashkeel
    text = normalizeArabic(text)
    
    text = re.sub('[A-Za-z]+',' ',text)
    
    text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    #Stemming may it is not important in the task!, for chractrize the dialects.
    #text = stem(text)
    return text

In [ ]:
!pip install flask-ngrok

In [ ]:
def load_embedding ():
    vec = pickle.load(open('/content/drive/MyDrive/vectorizer.pkl', 'rb'))
    return vec

In [ ]:
labels = {'IQ': 0, 'LY': 1, 'QA': 2, 'PL': 3, 'SY': 4
          , 'TN': 5, 'JO': 6, 'MA': 7, 'SA': 8, 'YE': 9, 'DZ': 10, 'EG': 11,
          'LB': 12, 'KW': 13, 'OM': 14, 'SD': 15, 'AE': 16, 'BH': 17}
langs = {v:k for k,v in labels.items()}

In [ ]:
def reduce (copus , vec , ii):
    X = vec.transform(copus)
    X = X[:, ii]
    X = coo_matrix(X)
    v = X.data
    i = np.vstack((X.row , X.col))
    v = torch.FloatTensor(v)
    i = torch.LongTensor(i)
    #tensor_X = torch.sparse_coo_tensor([X.row , X.col],X.data , dtype = torch.float)
    tensor_X = torch.sparse.FloatTensor(i , v, torch.Size(X.shape) ).to_dense()
    return tensor_X

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/DL" , methods = ['POST'])
def DL():
  data = request.json
  copus = [clean_text(text) for text in data]
  vec = load_embedding()
  ii = pickle.load(open('important_indexs.pkl', 'rb'))
  MLP_DL = MLP(2 ,len(ii) ,1000,18)
  MLP_DL.load_state_dict(torch.load('/content/drive/MyDrive/MLP4.pth'))
  MLP_DL.eval()
  X = reduce(copus , vec , ii)
  output = MLP_DL(X.float())
  _, predicted = torch.max(output.data, 1)
  labels = predicted.detach().numpy()
  out = [langs[l] for l in labels]
  return jsonify(out)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6e87-35-205-41-32.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
127.0.0.1 - - [10/Mar/2022 11:38:22] "POST //DL HTTP/1.1" 200 -
